# Solving partial differential equation-based Bayesian inverse problems using CUQIpy

* This demo is a modified version of https://github.com/CUQI-DTU/CUQIpy-demos/blob/main/training/Exercise05_PDE.ipynb

Here we build a Bayesian inverse problem in which the forward model is a partial differential equation (PDE) model, a 1D heat Bayesain inverse problem in particular.

##  1. Import required libraries

We first import the required python standard packages that we need:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

From CUQIpy we import the classes that we use in this demo:

In [ ]:
from cuqi.distribution import Gaussian, JointDistribution
from cuqi.geometry import Continuous1D, KLExpansion
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
from cuqi.sampler import CWMH
from cuqi.array import CUQIarray

## 2. Create the 1D heat PDE using the `TimeDependentLinearPDE` class 

### 2.1 The PDE model 
The PDE model is a one dimensional (1D) time-dependent heat PDE with zero boundary conditions.
The PDE is given by:

$$ \frac{\partial u(\xi,\tau)}{\partial t} - c^2 \Delta_\xi u(\xi,\tau)   = f(\xi,\tau), \;\text{in}\;[0,1] $$
$$u(0,\tau)= u(1,\tau)= 0 $$

where $u(\xi,\tau)$ is the temperature and $c^2$ is the thermal diffusivity (assumed to be 1 here). We assume the source term $f$ is zero.

The model is discretized using finite differences in space and forward Euler in time.

### 2.2 The inverse problem 

The unknown parameters  for the inverse problem we consider is the initial heat profile $g:=u(\xi,0)$. We infer $g$ from the data $y$: temperature measurements everywhere in the domain at the time $\tau^\text{max}$ (corrupted by Gaussian noise as measurement error).

We build the spatial grid for the 1D heat model

In [ ]:
n_grid = 100   # Number of solution nodes
h = 1/(n_grid+1)   # Space step size
grid = np.linspace(h, 1-h, n_grid)

Define the time steps

In [ ]:
tau_max = 0.01 # Final time
cfl = 5/11 # The cfl condition to have a stable solution
dt_approx = cfl*h**2 # Defining approximate time step size
n_tau = int(tau_max/dt_approx)+1 # Number of time steps
tau = np.linspace(0, tau_max, n_tau)

f, c and FD diffusion operator

In [ ]:
f = np.zeros(n_grid)
c = 1
D_c = c**2 * ( np.diag(-2*np.ones(n_grid), 0) +
np.diag(np.ones(n_grid-1), -1) +
np.diag(np.ones(n_grid-1), 1) ) / h**2

PDE form:

In [ ]:
def PDE_form(g, tau_current):
    return (D_c, f, g)

Create the `TimeDependentLinearPDE`

In [ ]:
PDE = TimeDependentLinearPDE(PDE_form, tau, grid_sol=grid)

In [ ]:
PDE

## 3. Create CUQIpy forward model using the `PDEModel` class

Set up geometries for model. We parameterize $g$:

$$ g = \sum_{i=1}^{N} x_i \phi_i(x) $$

Now $x_i$ are the parameters we want to characterize 

In [ ]:
# Domain geometry 
G_KL = KLExpansion(grid, decay_rate=1.5, normalizer=10, num_modes=20)

# Range geometry
G_cont = Continuous1D(grid)

 Prepare model

In [ ]:
A = PDEModel(PDE, range_geometry=G_cont, domain_geometry=G_KL)

## 4. Exact solution and exact data:

### 4.1 Set up the exact solution
The exact solution of the Bayesian inverse problem 

In [ ]:
x_custom =1/30*(1-np.cos(2*np.pi*(1-grid)/(1)))\
                +1/30*np.exp(-2*(10*(grid-0.5))**2)+\
                 1/30*np.exp(-2*(10*(grid-0.8))**2)

x_custom = CUQIarray(x_custom, is_par=False,
                    geometry=G_KL)

### 4.2 Generate the exact data

In [ ]:
y_custom = A(x=x_custom)

## 5. Set up the Bayesian inverse problem

The joint distribution of the data $y$ and the parameter $x$ is given by

$$p(x,y) = p(y|x)p(x)$$



Where $p(x)$ is the prior pdf, $p(y|x)$ is the data distribution pdf (likelihood). 

### 5.1 Create the prior distribution

Create a Gaussian prior distribution 

In [ ]:
x = Gaussian(np.zeros(G_KL.par_dim), 1, geometry=G_KL)
x

Now samples from the prior will look like:

In [ ]:
x_samples = x.sample(20)

plt.figure()
for s in x_samples:
    G_KL.plot(s, is_par=True)

### 5.2 Create the data distribution
To define the data distribution $p(y|x)$, we first estimate the noise level. Because here we know the exact data, we can estimate the noise level as follows:

In [ ]:
noise_level = 0.05
s_noise =1.0/np.sqrt(n_grid)* noise_level*np.linalg.norm(y_custom)

And then define the data distribution $p(y|x)$: 

In [ ]:
y = Gaussian(A(x), s_noise**2, geometry=G_cont)

### 5.3 Generate noisy data

In [ ]:
y_obs = y(x = x_custom).sample()

In [ ]:
plt.figure()
legend_list = []
x_custom.plot()
legend_list.append('exact solution')
y_custom.plot()
legend_list.append('exact data')
y_obs.plot()
legend_list.append('noisy data')
plt.legend(legend_list);

### 5.4 Specify the Bayesian model
Now that we have all the components we need, we can create the joint distribution $p(x,y)$, from which the posterior distribution can be created by setting $y=y_\text{obs}$:

First, we define the joint distribution $p(x,y)$:

In [ ]:
joint = JointDistribution(x, y)
joint

The posterior distribution pdf is given by the Bayes rule:
$$ p(x|y=y_\text{obs}) \propto p(y=y_\text{obs}|x)p(x) $$ 
By setting `y=y_obs` in the joint distribution we obtain the posterior distribution:

In [ ]:
posterior = joint(y=y_obs) # condition on y=y_obs
posterior

### 5.5 Specify the sampler and sample the posterior distribution 

In [ ]:
scale = np.ones(G_KL.par_dim)
scale[0] = 0.5

my_sampler = CWMH(posterior, scale = scale)

Sampling

In [ ]:
Ns = 50  
posterior_samples = my_sampler.sample_adapt(Ns)

### 5.6 Visualize the results 

Plot the parameter $95\%$ credible interval

In [ ]:
plt.figure()
posterior_samples.plot_ci(95, plot_par=True, exact=x_custom)

Plot the function values $95\%$ credible interval

In [ ]:
plt.figure()
posterior_samples.funvals.plot_ci(95, exact=x_custom)